# Data Preparation

An initial data quality analysis revealed structural, completeness, and consistency issues worth addressing. 

1.	The data are currently split into train and test sets by BI-RADS category. While convenient for modeling, this is rather cumbersome for the exploratory data analysis which considers the entire dataset, before making any assumptions. Our first task is to combine the calcification and mass train and test sets into a single case dataset, facilitating a top-down, comprehensive image view of all the data for analysis.
2.	The target variable, pathology, has three values: 'MALIGNANT', 'BENIGN', and 'BENIGN_WITHOUT_CALLBACK. The latter indicates that some aspect of the case is worth tracking; but, no follow-up is required. For our purposes, that is a distinction without a difference. Nonetheless, rather than modifying the variable and losing potentially useful information, we'll create a new target variable, 'cancer', which will be True if pathology is 'MALIGNANT', and False otherwise. 
3.	The CBIS-DDSM creators define a case as a particular abnormality as seen in the cranial-caudal (CC) or mediolateral oblique (MLO) image views; whereas, a DICOM series generally relates to a specific modality and may contain one or more DICOM objects (images). An ability to map an image to its case will be essential during the analysis stages; however, there is no way to uniquely identify a case. A case identifier will allow us to map cases to full mammogram, ROI, and cropped image series data.
4.	Developing an image preprocessing approach requires visibility into image data and pixel intensity statistics, which are currently buried in the individual DICOM files. These data must be extracted and then stored in a format that is accessible, and convenient. 
5.	We also encountered a corrupt image file. The DICOM image file for case_id 'P_01382_LEFT_mass_MLO_1' was corrupt, and could not be read. Therefore, the total image count is reduced to 10,238 images.
6.	The CBIS-DDSM has 45 calcification types, 9 calcification distributions, 20 mass shapes, and 19 mass margins, many of which are compound categories. For instance, calcification type 'ROUND_AND_REGULAR-PUNCTATE-AMORPHOUS' indicates three different types: 'ROUND_AND_REGULAR', 'PUNCTATE', and 'AMORPHOUS'. To distinguish the clinical and diagnostic importance of the morphological features, these data would be best represented as separate, dummy encoded variables. 

